# SoftmaxRegression

**Logostic regression**  
분류해야할 데이터들이 여러차원으로 표현 되어 있을 때  
이 데이터들을 구분하는 선을 찾는 과정

**Multinomial classification**
위 아이디어를 그대로 가져와서 여러가지 경우를 분류하는 것  
Binary Classification으로도 구현 가능  
예시)  
X -> A인지 아닌지 확힌하는 Classifier  
X -> B인지 아닌지 확힌하는 Classifier  
X -> C인지 아닌지 확힌하는 Classifier  
통과 시 세 그룹으로 분류 가능

이때 예시에서 각각 WX의 합을 구하여 분류하는 과정(Classifier통과)를 하나로 묶으면  
기존 $W X = w_1x_1 + w_2x_2 ... w_nx_n$과 같던 식이  
하나로 변함  
$$
\begin{bmatrix} 
w_{11} & w_{12} & w_{13} \\ 
w_{21} & w_{22} & w_{23} \\
w_{31} & w_{32} & w_{33}
\end{bmatrix} 
\begin{bmatrix} 
x_{1} \\ 
x_{2} \\ 
x_{3} 
\end{bmatrix} 
= \begin{bmatrix} 
w_{11}x_1 + w_{12}x_2 + w_{13}x_3 \\ 
w_{21}x_1 + w_{22}x_2 + w_{23}x_3 \\
w_{31}x_1 + w_{32}x_2 + w_{33}x_3
\end{bmatrix}
$$  
이때 하나하나의 행($w_{11}x_1 + w_{12}x_2 + w_{13}x_3$ 등)을 종합적으로 판단하기 위한 도구 필요  
따라서 시그모이드 함수 적용 후  
시그모이드의 을 다 받아서 총합이 1이 되는 값들로 변환  
이때 사용 되는 것이 SOFTMAX

$$ S(y_i) = \frac{e^{y_i}}{\sum_j e^{y_j}}$$  
여기서 나온 값을 받아 argmax를 적용(OneHotEncoding)해줌으로서 분류 완료  
$$
Y
\begin{bmatrix}
2.0 \\
1.0 \\
0.1 
\end{bmatrix}
\rightarrow
S(y)
\rightarrow
P
\begin{bmatrix}
0.7 \\
0.2 \\
0.1 
\end{bmatrix}
\rightarrow
argmax
\rightarrow
L
\begin{bmatrix}
1 \\
0 \\
0 
\end{bmatrix}
$$

**Cost Function**
**Cross-Entropy**  
S(y) 와 L을 사용한 손실함수  
$$D(S,L) = -\sum_{i} L_i \log(S_i)$$

-log는 0과 1사이의 값만 받을 수 있고  
이는 통과 시 틀린 정답에 대해 강력한 손실값을 반환해 주고  
맞는 정답에 대해서는 손실값으로 0반환  
따라서 손실함수로서 적합

지난 챕터의 Logistc Cost  
$c(H(x), y) = -y\log(H(x)) -(1-y)\log(1-H(x))$ 와 구조적으로 동일함  

In [1]:
import tensorflow as tf

tf.random.set_seed(777)

x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]

y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

In [3]:
x_train = tf.cast(x_data, tf.float32)
y_train = tf.cast(y_data, tf.float32)

nb_classes = 3

W = tf.Variable(tf.random.normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random.normal([nb_classes]), name='bias')

optimizer = tf.optimizers.SGD(learning_rate=0.1)

def hypothesis(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

입력값에 가중치를 곱하고 편향을 더한 계산값이  
softmax를 통과하며 최종적으로 내놓는 답은 확률로 표현 됨

In [4]:
def cost_fn(X, Y):
    logits = hypothesis(X)
    cost_i = tf.reduce_sum(Y * tf.math.log(logits + 1e-7), axis=1)
    return tf.reduce_mean(-cost_i)

$D(S,L) = -\sum_{i} L_i \log(S_i)$에 H(x)값 넘겨 주는 것 구현

In [5]:
for step in range(2001):
    with tf.GradientTape() as tape:
        curr_cost = cost_fn(x_train, y_train)

    grads = tape.gradient(curr_cost, [W, b])
    optimizer.apply_gradients(zip(grads, [W, b]))

    if step % 200 == 0:
        print(f'{step}\t{curr_cost.numpy():.6f}')

0	4.456855
200	0.524867
400	0.416868
600	0.335701
800	0.261106
1000	0.229923
1200	0.208102
1400	0.190058
1600	0.174857
1800	0.161866
2000	0.150633


학습 체크

In [6]:
test_data = tf.cast([[1, 11, 7, 9]], tf.float32)
a = hypothesis(test_data)
print(f"{a.numpy()}\nPredict: {tf.argmax(a, axis=1).numpy()}") 

test_data = tf.cast([[1, 3, 4, 3]], tf.float32)
b_pred = hypothesis(test_data)
print(f"{b_pred.numpy()}\nPredict: {tf.argmax(b_pred, axis=1).numpy()}") 

test_data = tf.cast([[1, 1, 0, 1]], tf.float32)
c = hypothesis(test_data)
print(f"{c.numpy()}\nPredict: {tf.argmax(c, axis=1).numpy()}") 

all_pred = hypothesis(x_train)
print(f"{all_pred.numpy()}\nPredict: {tf.argmax(all_pred, axis=1).numpy()}")

[[1.3327835e-03 9.9865854e-01 8.6538557e-06]]
Predict: [1]
[[0.93171006 0.0628916  0.00539825]]
Predict: [0]
[[1.0695679e-08 3.2634559e-04 9.9967372e-01]]
Predict: [2]
[[3.91332151e-06 1.05971191e-03 9.98936355e-01]
 [2.99250358e-03 8.31682459e-02 9.13839221e-01]
 [2.79758083e-08 1.61043167e-01 8.38956773e-01]
 [8.45727811e-07 8.51096332e-01 1.48902804e-01]
 [2.43110850e-01 7.44687140e-01 1.22020217e-02]
 [1.27047539e-01 8.72940838e-01 1.16033225e-05]
 [7.68399298e-01 2.31578097e-01 2.26548218e-05]
 [9.20781970e-01 7.92177096e-02 3.84079527e-07]]
Predict: [2 2 2 1 1 1 0 0]


**Fancy Softmax Classifier**